In [ ]:
import pandas as pd

In [ ]:
# We have two sheets in an excel file, one for california and one for nevada
# We want to combine them into one dataframe
# We need to standardize a traffic count station feature class
traffic_data = "data/AADT_JS_Updated20240312.xlsx"
ca_data = pd.read_excel(traffic_data, sheet_name='CA')
nv_data = pd.read_excel(traffic_data, sheet_name='NV')
all_data = pd.concat([ca_data, nv_data], ignore_index=True)